In [70]:
"""
Implementation of AlexNet, from paper
"ImageNet Classification with Deep Convolutional Neural Networks" by Alex Krizhevsky et al.
See: https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf
"""
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [71]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 10),        # 10 output classes
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x
model=AlexNet()
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=1024, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, in

In [72]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):

    train_losses = []
    valid_losses = []
    best_model_loss = 5
    
    for epoch in range(num_epochs):

        # Training
        for i, (data, labels) in enumerate(train_loader):
      
            prediction = model.forward(data)

            train_loss = criterion(prediction, labels)

            train_loss.backward()

            optimizer.step()

            optimizer.zero_grad()
        print(f'\rEpoch {epoch+1}, batch {i+1}/{len(train_loader)} - Loss: {train_loss}')

        train_losses.append(train_loss)
        writer.add_scalar("Loss/train ADAM", train_loss, epoch)

        # Validation
        for batch_nr, (data, labels) in enumerate(val_loader):
            prediction = model.forward(data)
            loss_val = criterion(prediction, labels)
            valid_losses.append(loss_val)
        print(f"loss validation: {loss_val}")
        #print(f"loss validation: {loss_val}","\n")

        if valid_losses[-1] < best_model_loss:
            print(f"\t > Found a better model, {best_model_loss} -> {valid_losses[-1]}")
            best_model = copy.deepcopy(model)
            best_model_loss = valid_losses[-1]

        writer.add_scalar("Loss/validation ADAM", loss_val, epoch)

    print(f"\nBest model loss: {best_model_loss}")
    return best_model, train_losses, valid_losses

def get_accuracy(network, loader):
    
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        y_true = []

        for x, (data, labels) in enumerate(loader):

            prediction = network.forward(data)

            for i in range(len(data)):

                y_true.append(labels[i].item())
                y_pred.append(torch.argmax(prediction[i]).item())
                if y_true[i] == y_pred[i]:
                    correct += 1        
    
            total += float(len(data))
    
        score = correct/total

        accuracy = score

        return accuracy
    

In [73]:
LEARNING_RATE = 0.001
EPOCHS = 5
BATCH_SIZE = 1000
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

validset, trainset = torch.utils.data.random_split(trainset, [10000, 40000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)



# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model, train_loss, valid_loss = train_model(model, criterion, optimizer, trainloader, validloader, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model, testloader)
print(f"Model Accuracy (CIFAR10): {test_acc*100}%")
writer.flush()

Files already downloaded and verified
Files already downloaded and verified
Epoch 1, batch 40/40 - Loss: 2.002079725265503
loss validation: 1.9644508361816406
	 > Found a better model, 5 -> 1.9644508361816406
Epoch 2, batch 40/40 - Loss: 1.637309193611145
loss validation: 1.598850965499878
	 > Found a better model, 1.9644508361816406 -> 1.598850965499878
Epoch 3, batch 40/40 - Loss: 1.3502089977264404
loss validation: 1.4216865301132202
	 > Found a better model, 1.598850965499878 -> 1.4216865301132202
Epoch 4, batch 40/40 - Loss: 1.3123183250427246
loss validation: 1.2828093767166138
	 > Found a better model, 1.4216865301132202 -> 1.2828093767166138
Epoch 5, batch 40/40 - Loss: 1.0924111604690552
loss validation: 1.131852626800537
	 > Found a better model, 1.2828093767166138 -> 1.131852626800537

Best model loss: 1.131852626800537
Model Accuracy (CIFAR10): 56.8%


In [74]:
model = nn.Sequential(
    model,
    nn.Linear(10, 10))
print(model)

Sequential(
  (0): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=1024, out_features=4096, bias=True)

In [75]:
LEARNING_RATE = 0.01
EPOCHS = 5
num_classes=10
model_featext=trained_model


# Freeze all layers except the last few layers
for name, param in model_featext.named_parameters():
    if "Sequential.1" in name or "classifier.6" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model_featext, train_loss, valid_loss = train_model(model_featext, criterion, optimizer, trainloader, validloader, EPOCHS)


test_acc = get_accuracy(trained_model_featext, testloader)
print(f"Model Accuracy (AlexNet PreTrained): {test_acc*100}%")
writer.flush()

Epoch 1, batch 40/40 - Loss: 1.0360560417175293
loss validation: 1.1474992036819458
	 > Found a better model, 5 -> 1.1474992036819458
Epoch 2, batch 40/40 - Loss: 1.1284067630767822
loss validation: 1.1342777013778687
	 > Found a better model, 1.1474992036819458 -> 1.1342777013778687
Epoch 3, batch 40/40 - Loss: 1.0990573167800903
loss validation: 1.191811203956604
Epoch 4, batch 40/40 - Loss: 1.0771840810775757
loss validation: 1.163754940032959
Epoch 5, batch 40/40 - Loss: 1.0282353162765503
loss validation: 1.1006598472595215
	 > Found a better model, 1.1342777013778687 -> 1.1006598472595215

Best model loss: 1.1006598472595215
Model Accuracy (AlexNet PreTrained): 58.8%
